<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import talos as ta
from google.colab import drive
# Load CIFAR-100 dataset
# Mount Google Drive
drive.mount("/content/drive")
def load_data():

# Load CIFAR-100 dataset from Google Drive
    with open('/content/drive/My Drive/train', 'rb') as file:
        train_dict = pickle.load(file, encoding='bytes')

    with open('/content/drive/My Drive/test', 'rb') as file:
        test_dict = pickle.load(file, encoding='bytes')

    X_train = train_dict[b'data']
    y_train = train_dict[b'coarse_labels']
    X_test = test_dict[b'data']
    y_test = test_dict[b'coarse_labels']

    enc = OneHotEncoder(sparse=False, categories='auto')
    y_train = enc.fit_transform(np.array(y_train).reshape(-1, 1))
    y_test = enc.transform(np.array(y_test).reshape(-1, 1))

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    return X_train, y_train, X_test, y_test

# Define the neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_units, hidden_activation, output_size):
        super(NeuralNet, self).__init__()
        activation_fn = nn.ReLU() if hidden_activation == 'relu' else nn.Sigmoid()

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            activation_fn,
            nn.Linear(hidden_units, hidden_units),
            activation_fn,
            nn.Linear(hidden_units, hidden_units),
            activation_fn,
            nn.Linear(hidden_units, hidden_units),
            activation_fn,
            nn.Linear(hidden_units, output_size),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.layers(x)

# Train function
def train_model(X_train, y_train, X_test, y_test, params):
    model = NeuralNet(3072, params['units'], params['hidden_activations'], 100)
    criterion = nn.CrossEntropyLoss() if params['loss'] == 'categorical_crossentropy' else nn.MSELoss()
    optimizer = optim.Adam(model.parameters()) if params['optimizer'] == 'adam' else optim.Adagrad(model.parameters())

    train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), batch_size=params['batch_size'], shuffle=True)
    test_loader = data.DataLoader(data.TensorDataset(X_test, y_test), batch_size=params['batch_size'], shuffle=False)

    history = {'epoch': [], 'accuracy': []}
    best_acc = 0.0

    for epoch in range(200):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                outputs = model(batch_X)
                _, predicted = torch.max(outputs, 1)
                _, labels = torch.max(batch_y, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        accuracy = correct / total
        history['epoch'].append(epoch)
        history['accuracy'].append(accuracy)

        if accuracy > best_acc:
            best_acc = accuracy

    print(f"Best Accuracy: {best_acc * 100:.2f}%")

    df = pd.DataFrame(history)
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x='epoch', y='accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Test Accuracy')
    plt.title('CIFAR-100 Training Results')
    plt.grid()
    plt.savefig("/content/drive/My Drive/cifar100_training_plot.png", dpi=300)
    plt.show()

    return model

# Define Talos parameter dictionary
p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

# Load data
X_train, y_train, X_test, y_test = load_data()

# Run Talos scan
ta.Scan(x=X_train.numpy(),
        y=y_train.numpy(),
        model=train_model,
        params=p,
        experiment_name='cifar100_talos')

IndentationError: expected an indented block after function definition on line 14 (<ipython-input-1-2dfd451f4191>, line 18)